<a href="https://colab.research.google.com/github/Elhassnaoui2001/Ecommerce/blob/Elhassnaoui2001-patch-1/starter_notebook_Zindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nowcasting Starter Notebook

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In this notebook, a dataset containing new vehicle sales (Naamsa) is used to predict the CPI values. This notebook only contains one simple linear regression model, however, you are tasked to predict all the sub-categories of CPI as well as the overall CPI value. Therefore, you potentially could have 13 models. The notebook also contains a simple way to predict sub-category CPI values using historic values.

#### Step 1: Import Libraries

In programming, we often use pre-built libraries that contain useful functions and tools for our specific task. In the code snippet below are examples containing such library imports.

The first library, pandas, provides tools for working with structured data, while the second library, numpy, provides powerful numerical computing tools. The third library, matplotlib, allows us to create visualizations of our data, and the fourth library, datetime, provides functions for working with dates and times. Lastly, we are importing specific functions from the scikit-learn library, including train_test_split for splitting our data into training and testing sets, mean_squared_error for evaluating the performance of our model, LinearRegression for fitting a linear regression model, and StandardScaler for standardizing our data. By importing these libraries, we have access to a range of useful tools and functions that will help us in our programming task.

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

#### Import the dataset

You should change the name: "New_Vehicle_Sales_(Naamsa).csv" to your filepath of your file.

In [102]:
df = pd.read_csv("/content/drive/MyDrive/RMB Monthly Mini Challenge #4: Nowcast July/Naamsa_Vehicle_Sales.csv")
cpi_values = pd.read_csv("/content/drive/MyDrive/RMB Monthly Mini Challenge #4: Nowcast July/CPI_Historic_Values_Zindi_Feb_23.csv")
df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)

<ipython-input-102-0c2fb21fc88a>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)


Add the dates to the dataset.

In [103]:
# start date
start_date = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2023-03-01", "%Y-%m-%d")

# difference between each date. MS means one month
D = 'MS'

date_list = pd.date_range(start_date, end_date, freq=D)[::-1]

Rename the Month column in the dataframe to Date

In [104]:
df['Month'] = date_list
df = df.rename(columns={'Month': 'Date'})

Change the Date column values to datetime format so that Python can understand that it is a date value.

In [105]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%b-%d')

Create a filter to filter only for a certain date range as the values in the CPI Historic are only in this range.

In [106]:
df = df[(df['Date'] < '2023-03-01') & (df['Date'] >= '2022-01-01') ]
df['CPI'] = cpi_values.loc[cpi_values['Category'] == 'headline_CPI', 'Value'].tolist()

ValueError: ignored

We can plot how the series has changed over time. The library we are using here is ggplot.

In [ ]:
plt.plot(df['Date'], df['CPI'], linewidth=2)
plt.title('CPI South Africa')
plt.xlabel('Date')
plt.ylabel('Value')
plt.xticks(rotation = 30)

A linear regression model unfortunately does not work with the Date as an object, therefore, we can use this line to convert it to a numerical value for the linear regression model.

In [ ]:
df['Date']=df['Date'].map(datetime.datetime.toordinal)

In machine learning, a training set is a subset of data used to train a model, while a test set is a subset of data used to evaluate the performance of the trained model on new, unseen data. The training set is used to fit the model parameters, while the test set is used to estimate how well the model generalizes to new data. Here we can split the dataset into a training and test set with a ratio of 80:20.

In [ ]:
y = df['CPI']
X = df.drop('CPI', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

Finally, we can fit our model using our training set.

In [ ]:
# model = LinearRegression()
# model.fit(X_train, y_train)

We can use the predict attribute to predict the CPI for the test set.

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)


In [ ]:
predictions = ridge.predict(X_test)

We can use the mean squared error function from the sklearn package to calculate the RMSE of the model.

In [ ]:
mean_squared_error(y_test,predictions)

### Training the model on historical data and then predicting February 2023 CPI

Import dataset and change Date column to datetime object

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/RMB Monthly Mini Challenge #4: Nowcast July/Naamsa_Vehicle_Sales.csv")
cpi_values = pd.read_csv("/content/drive/MyDrive/RMB Monthly Mini Challenge #4: Nowcast July/CPI_Historic_Values_Zindi_Feb_23.csv")
df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)

In [ ]:
# start date
start_date = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2023-03-01", "%Y-%m-%d")

# difference between each date. MS means one motnh
D = 'MS'

date_list = pd.date_range(start_date, end_date, freq=D)[::-1]

In [ ]:
df['Month'] = date_list
df = df.rename(columns={'Month': 'Date'})
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%b-%d')
df = df[(df['Date'] < '2023-03-01') & (df['Date'] >= '2022-01-01') ]
df['CPI'] = cpi_values.loc[cpi_values['Category'] == 'headline_CPI', 'Value'].tolist()

Remove the data for February from the dataset

In [ ]:
mask = (df["Date"] == "2023-02-01")
february = df[mask]
df = df[~mask]

Splitting remaining data into a train and test set

In [ ]:
X = df.drop('CPI', axis=1)
y = df['CPI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)
X_train['Date']=X_train['Date'].map(datetime.datetime.toordinal)
X_test['Date']=X_test['Date'].map(datetime.datetime.toordinal)
february['Date'] = february['Date'].map(datetime.datetime.toordinal)

Fit the model using the training data

In [ ]:
# model = LinearRegression()
# model.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)


Use the model to predict the February CPI value and calculate RMSE

In [ ]:
predictions = ridge.predict(february.drop(columns=['CPI']))
mean_squared_error(february['CPI'],predictions)

## Sub-category prediction model example:

The headline CPI value consists of a weighted sum of 12 sub-categories. Therefore, a prediction should be made for each of the 12 sub-categories. For this first starter notebook, a simple weighted calculation will be used to calculate a given sub-category CPI value.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/RMB Monthly Mini Challenge #4: Nowcast July/CPI_Historic_Values_Zindi.csv')
cpi_weights = pd.read_excel('/content/drive/MyDrive/RMB Monthly Mini Challenge #4: Nowcast July/cpi_weights.xlsx')
categories = df['Category'].unique()
df['Month'] = pd.to_datetime(df['Month'], format='%d-%m-%Y')
df.sort_values(by='Month', inplace = True, ascending= False)

Obtain the most recent cpi and sub-category values by creating a filter. This code snippet produces all the data from January 2023.

In [ ]:
recent_cpi_values = df.loc[df['Month'] >= '01-01-2023']

The following code uses a weighting system, giving most weight to the most recent sub-category CPI values, to predict the change in the CPI values for February:

In [ ]:
cpi_dict = {}

for category in df['Category'].unique():
    filtered_df = df[df['Category'] == category].copy()

    filtered_df['Month'] = pd.to_datetime(filtered_df['Month'], format='%d-%m-%Y')
    filtered_df.sort_values(by='Month', inplace=True, ascending=False)

    weights = 0.5 * pd.Series([0.5**i for i in range(len(filtered_df))])

    filtered_df['Weighted Percentage Change'] = weights.values * filtered_df['Percentage Change (From Prior Month)']
    cpi_value = filtered_df['Weighted Percentage Change'].sum()
    cpi_dict[category] = cpi_value
    if category != 'Headline_CPI':
        print(f'February weighted sub-category CPI percentage change for {category} is: {round(cpi_value,2)}')

We can now calculate the Headline CPI values now that we predicted the change in each sub-category CPI value. We combine the weights from StatsSA and the previous months sub-category CPI value to calculate the Headline CPI value for February:

In [ ]:
weighted_cpi_value = 0
for category in cpi_dict.keys():
    weight = cpi_weights.loc[cpi_weights['Category'] == category, 'Weight'].values[0]
    change = recent_cpi_values.loc[recent_cpi_values['Category'] == category, 'Value'].values[0]
    cpi_value = (cpi_dict[category] + change) * weight/100
    if category != 'Headline_CPI':
        weighted_cpi_value += cpi_value
cpi_dict['Headline_CPI'] = round(weighted_cpi_value, 2)
print(f'The predicted February value for the Headline CPI is {round(weighted_cpi_value,2)}.')

We follow the same idea to predict the sub-category CPI values for March by using an extrapolated the results from February.

In [ ]:
cpi_dict = {}
cpi_percentage_change = {}
for category in df['Category'].unique():
    filtered_df = df[df['Category'] == category].copy()

    filtered_df['Month'] = pd.to_datetime(filtered_df['Month'], format='%d-%m-%Y')
    filtered_df.sort_values(by='Month', inplace=True, ascending=False)

    weights = 0.5 * pd.Series([0.5**i for i in range(len(filtered_df))])

    filtered_df['Weighted Percentage Change'] = weights.values * filtered_df['Percentage Change (From Prior Month)'] * 2
    cpi_value = filtered_df['Weighted Percentage Change'].sum()
    cpi_dict[category] = cpi_value
    if category != 'Headline_CPI':
        print(f'March weighted sub-category CPI percentage change for {category} is: {round(cpi_value,2)}')

Here we again calculate the Headline CPI value for March by using the weights from StatsSA.

In [ ]:
weighted_cpi_value = 0
for category in cpi_dict.keys():
    weight = cpi_weights.loc[cpi_weights['Category'] == category, 'Weight'].values[0]
    change = recent_cpi_values.loc[recent_cpi_values['Category'] == category, 'Value'].values[0]
    cpi_value_new = cpi_dict[category] + change
    cpi_value_weighted = cpi_value_new * weight/100
    cpi_dict[category] = round(cpi_value_new, 2)
    if category != 'Headline_CPI':
        weighted_cpi_value += cpi_value_weighted
cpi_dict['Headline_CPI'] = round(weighted_cpi_value, 2)
print(f'The predicted March value for the headline CPI is {round(weighted_cpi_value,2)}.')

We follow the same idea to predict the sub-category CPI values for April by using an extrapolated the results from February.

In [ ]:
cpi_dict = {}
cpi_percentage_change = {}
for category in df['Category'].unique():
    filtered_df = df[df['Category'] == category].copy()

    filtered_df['Month'] = pd.to_datetime(filtered_df['Month'], format='%d-%m-%Y')
    filtered_df.sort_values(by='Month', inplace=True, ascending=False)

    weights = 0.5 * pd.Series([0.5**i for i in range(len(filtered_df))])

    filtered_df['Weighted Percentage Change'] = weights.values * filtered_df['Percentage Change (From Prior Month)'] * 3
    cpi_value = filtered_df['Weighted Percentage Change'].sum()
    cpi_dict[category] = cpi_value
    if category != 'Headline_CPI':
        print(f'April weighted sub-category CPI percentage change for {category} is: {round(cpi_value,2)}')

Lastly, we calculate the Headline CPI value for April.

In [ ]:
weighted_cpi_value = 0
for category in cpi_dict.keys():
    weight = cpi_weights.loc[cpi_weights['Category'] == category, 'Weight'].values[0]
    change = recent_cpi_values.loc[recent_cpi_values['Category'] == category, 'Value'].values[0]
    cpi_value_new = cpi_dict[category] + change
    cpi_value_weighted = cpi_value_new * weight/100
    cpi_dict[category] = round(cpi_value_new, 2)
    if category != 'Headline_CPI':
        weighted_cpi_value += cpi_value_weighted

cpi_dict['headline_CPI'] = round(weighted_cpi_value, 2)
print(f'The predicted July value for the headline CPI is {round(weighted_cpi_value,2)}.')

Now to get the results ready for submission:

In [ ]:
cpi_predictions = pd.DataFrame(list(cpi_dict.items()), columns=['ID', 'Value'])
cpi_predictions['ID'] = 'July_' + cpi_predictions['ID']

Output to excel file:

In [ ]:
cpi_predictions.to_csv('ridge.csv', index = 0)